In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import syft as sy
import copy
import numpy as np
import time

import importlib
importlib.import_module('FLDataset')
from FLDataset import load_dataset, getActualImgs
from utils import averageModels, normalModels

In [2]:
class Arguments():
    def __init__(self):
        self.images = 50000
        self.clients = 20
        self.rounds = 10
        self.epochs = 5
        self.local_batches = 64
        self.lr = 0.01
        self.C = 1.0
        self.drop_rate = 0.0
        self.torch_seed = 0
        self.log_interval = 100
        self.iid = 'iid'
        self.split_size = int(self.images / self.clients)
        self.samples = self.split_size / self.images 
        self.use_cuda = False
        self.save_model = False

args = Arguments()

use_cuda = args.use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [3]:
writer = open('20_experiments_cifar.csv', 'a')
writer.write('\n\n\n')
writer.write('Clients {}, Rounds {}, Epochs {}, IID {}'.format(args.clients, args.rounds, args.epochs, args.iid))
writer.write('\n')

1

In [4]:
hook = sy.TorchHook(torch)
clients = []

for i in range(args.clients):
    clients.append({'hook': sy.VirtualWorker(hook, id="client{}".format(i+1))})

In [5]:
# # Download MNIST manually using 'wget' then uncompress the file
# !wget www.di.ens.fr/~lelarge/MNIST.tar.gz
# !tar -zxvf MNIST.tar.gz

In [6]:
global_train, global_test, train_group, test_group = load_dataset(args.clients, args.iid)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
for inx, client in enumerate(clients):
    trainset_ind_list = list(train_group[inx])
    client['trainset'] = getActualImgs(global_train, trainset_ind_list, args.local_batches)
    client['testset'] = getActualImgs(global_test, list(test_group[inx]), args.local_batches)
#     client['samples'] = len(trainset_ind_list) / args.images
    client['samples'] = len(trainset_ind_list) / (len(trainset_ind_list) * args.clients)

In [8]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
global_test_dataset = datasets.CIFAR10('./', train=False, download=True, transform=transform)
global_test_loader = DataLoader(global_test_dataset, batch_size=args.local_batches, shuffle=True)

Files already downloaded and verified


In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [10]:
def ClientUpdate(args, device, client):
    client['model'].train()
#     client['model'].send(client['hook'])
    
    for epoch in range(1, args.epochs + 1):
        for batch_idx, (data, target) in enumerate(client['trainset']):
#             data = data.send(client['hook'])
#             target = target.send(client['hook'])
            
            data, target = data.to(device), target.to(device)
            client['optim'].zero_grad()
            output = client['model'](data)
            loss = F.nll_loss(output, target)
            loss.backward()
            client['optim'].step()
            
            if batch_idx % args.log_interval == 0:
#                 loss = loss.get() 
                print('Model {} Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    client['hook'].id,
                    epoch, batch_idx * args.local_batches, len(client['trainset']) * args.local_batches, 
                    100. * batch_idx / len(client['trainset']), loss))
                
#     client['model'].get() 

In [11]:
def test(args, model, device, test_loader, name):
    model.eval()   
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss for {} model: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        name, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    writer.write('{},'.format(100. * correct / len(test_loader.dataset)))

In [12]:
def Aggregate(all_models, sample_client):
    all_dicts = {}
    for inds, client in enumerate(all_models):
        all_dicts[client['hook'].id] = all_models[inds]['model'].state_dict()
    output_dict = copy.deepcopy(all_dicts[sample_client])
    for key in all_dicts[sample_client]:
        for val in all_dicts:
            if val == sample_client:
                continue
            output_dict[key] += all_dicts[val][key]
#         output_dict[key] /= float(len(all_dicts))
        output_dict[key] /= float(len(all_models))
    output_model = Net().to(device)
    output_model.load_state_dict(output_dict)
    return output_model

In [13]:
torch.manual_seed(args.torch_seed)
global_model = Net()
# torch.manual_seed(args.torch_seed)
# normal_avg = Net()
# torch.manual_seed(args.torch_seed)
# fake_model = Net()

for client in clients:
    torch.manual_seed(args.torch_seed)
    client['model'] = Net().to(device)
    client['optim'] = optim.SGD(client['model'].parameters(), lr=args.lr)

start_time = time.time()

for fed_round in range(args.rounds):
    
#     uncomment if you want a randome fraction for C every round
#     args.C = float(format(np.random.random(), '.1f'))
    
    # number of selected clients
    m = int(max(args.C * args.clients, 1))

    # Selected devices
    np.random.seed(fed_round)
    selected_clients_inds = np.random.choice(range(len(clients)), m, replace=False)
    selected_clients = [clients[i] for i in selected_clients_inds]
    
    # Active devices
    np.random.seed(fed_round)
    active_clients_inds = np.random.choice(selected_clients_inds, int((1-args.drop_rate) * m), replace=False)
    active_clients = [clients[i] for i in active_clients_inds]
    
    # Training 
    for client in active_clients:
        ClientUpdate(args, device, client)
    
#     # Testing 
#     for client in active_clients:
#         test(args, client['model'], device, client['testset'], client['hook'].id)
    
    # Averaging 
    global_model = averageModels(global_model, active_clients)
#     normal_avg = normalModels(normal_avg, active_clients)
#     fake_model = Aggregate(active_clients, 'client1')
    
    # Testing the average model
    test(args, global_model, device, global_test_loader, 'Global')
#     test(args, normal_avg, device, global_test_loader, 'Global')
#     test(args, fake_model, device, global_test_loader, 'Global')
    
#     break
            
    # Share the global model with the clients
    for client in clients:
        client['model'].load_state_dict(global_model.state_dict())

writer.write('\n')
end_time = time.time() 
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
writer.write('{:0>2}:{:0>2}:{:05.2f}'.format(int(hours),int(minutes),seconds))
writer.write('\n')
writer.close()

if (args.save_model):
    torch.save(global_model.state_dict(), "FedAvg.pt")

/Users/gharibi/opt/anaconda3/envs/iswc/lib/python3.7/site-packages/syft/frameworks/torch/hook/hook.py:560: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  current_tensor = hook_self.torch.native_tensor(*args, **kwargs)


Model client16 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.323728
Model client16 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.300640
Model client16 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.319030
Model client16 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.310430
Model client16 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.302511
Model client2 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.322279
Model client2 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.292531
Model client2 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.299138
Model client2 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.310438
Model client2 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.306267
Model client13 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.307345
Model client13 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.318027
Model client13 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.306035
Model client13 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.301256
Model client13 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.298143
Model client5 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.300318
Model client5 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.306651
Mode

Model client13 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.296944
Model client18 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.299235
Model client18 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.304365
Model client18 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.296777
Model client18 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.305359
Model client18 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.294966
Model client17 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.307634
Model client17 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.294502
Model client17 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.301046
Model client17 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.299839
Model client17 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.305859
Model client19 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.298034
Model client19 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.298438
Model client19 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.307295
Model client19 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.294428
Model client19 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.289857
Model client4 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.30256

Model client4 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.295384
Model client4 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.298697
Model client9 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.298023
Model client9 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.296778
Model client9 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.294650
Model client9 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.303546
Model client9 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.292366
Model client3 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.291334
Model client3 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.301196
Model client3 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.286659
Model client3 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.306181
Model client3 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.304317
Model client7 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.307532
Model client7 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.302808
Model client7 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.294711
Model client7 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.295108
Model client7 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.299205
Model client2 

Model client1 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.280632
Model client1 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.277520
Model client1 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.284501
Model client1 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.266075
Model client3 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.281185
Model client3 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.282131
Model client3 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.277183
Model client3 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.275913
Model client3 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.259233
Model client14 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.292901
Model client14 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.290032
Model client14 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.281552
Model client14 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.282814
Model client14 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.270793
Model client20 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.298458
Model client20 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.287085
Model client20 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.279637
Model 

Model client13 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.266217
Model client13 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.256933
Model client13 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.258492
Model client13 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.207960
Model client13 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.199771
Model client14 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.269571
Model client14 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.258323
Model client14 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.262989
Model client14 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.237367
Model client14 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.227782
Model client19 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.252970
Model client19 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.254649
Model client19 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.244196
Model client19 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.232434
Model client19 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.215644
Model client3 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.257390
Model client3 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.259625

Model client12 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.080188
Model client13 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.131945
Model client13 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.114577
Model client13 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.101383
Model client13 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.032755
Model client13 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.090014

Test set: Average loss for Global model: 2.0276, Accuracy: 2829/10000 (28%)

Model client18 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.874396
Model client18 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.097371
Model client18 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.978651
Model client18 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.962360
Model client18 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.874481
Model client4 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.947435
Model client4 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.010498
Model client4 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.922888
Model client4 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.068801
Model client4 Train Epoch: 5 [0/2560 (0%)

Model client15 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.299998
Model client15 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.986341
Model client15 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.866622
Model client7 Train Epoch: 1 [0/2560 (0%)]	Loss: 2.068146
Model client7 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.095747
Model client7 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.061277
Model client7 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.856809
Model client7 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.866251
Model client20 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.938182
Model client20 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.058139
Model client20 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.985029
Model client20 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.467787
Model client20 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.842880
Model client12 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.991158
Model client12 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.089877
Model client12 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.983422
Model client12 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.963930
Mo

Model client15 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.182929
Model client15 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.994376
Model client15 Train Epoch: 4 [0/2560 (0%)]	Loss: 2.042404
Model client15 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.078709
Model client1 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.845926
Model client1 Train Epoch: 2 [0/2560 (0%)]	Loss: 2.212697
Model client1 Train Epoch: 3 [0/2560 (0%)]	Loss: 1.821404
Model client1 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.796364
Model client1 Train Epoch: 5 [0/2560 (0%)]	Loss: 1.983367
Model client2 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.919272
Model client2 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.990485
Model client2 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.037530
Model client2 Train Epoch: 4 [0/2560 (0%)]	Loss: 1.900459
Model client2 Train Epoch: 5 [0/2560 (0%)]	Loss: 2.153669
Model client18 Train Epoch: 1 [0/2560 (0%)]	Loss: 1.952517
Model client18 Train Epoch: 2 [0/2560 (0%)]	Loss: 1.982730
Model client18 Train Epoch: 3 [0/2560 (0%)]	Loss: 2.004982
Model c